## Загрузка датасета

In [1]:
!pip install kaggle
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!echo '{"username":"midlow","key":"19e4a7b3c26e4d040a5179c6b36318cd"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d dhruvildave/wikibooks-dataset
!unzip wikibooks-dataset.zip
!rm wikibooks-dataset.zip
!rm *.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
rm: cannot remove '/root/.kaggle': No such file or directory
 99% 1.81G/1.82G [00:10<00:00, 204MB/s]
100% 1.82G/1.82G [00:10<00:00, 181MB/s]
Archive:  wikibooks-dataset.zip
  inflating: wikibooks.sqlite        
rm: cannot remove '*.zip': No such file or directory


In [2]:
import pandas as pd
import numpy as np
import re

In [11]:
import sqlite3

con = sqlite3.connect("wikibooks.sqlite")
df = pd.read_sql_query("SELECT body_text FROM en limit 1000", con)
df.head()

,body_text
0,Front Page: Radiation Oncology | RTOG Trials |...
1,Băuturi/Beverages[edit | edit source]\nTea : C...
2,Karrigell is an open Source Python web framewo...
3,setupUnitPanel[edit | edit source]\nHelper fun...
4,Contents\n\n1 The Concept\n2 The System\n3 The...


In [12]:
def cleaning(s):
    s = str(s)
    s = re.sub(r'[^a-zA-Z0-9 ]', ' ', s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace("[\w*"," ")
    return s

In [13]:
text_data = open('Wikipages.txt', 'w')
for idx, item in df.iterrows():
  article = cleaning(item["body_text"])
  text_data.write(article)
text_data.close()

## Тренировка модели

In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.3 MB/s eta 0:00:00


In [6]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.7 MB/s eta 0:00:00


In [15]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [16]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator


def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

In [19]:
train_file_path = "Wikipages.txt"
model_name = 'gpt2'
output_dir = 'result'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 3.0
save_steps = 500

In [20]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

Step,Training Loss
500,4.457000
1000,4.263800
1500,4.174900
2000,4.159900
2500,4.087300
3000,4.085600
3500,4.046600
4000,4.040800
4500,3.992800
5000,4.012800


## Взаимодействие с обученной моделью

In [21]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [22]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "result"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [24]:
sequence = "Coronovirus is a strong desease"
max_len = 100
generate_text(sequence, max_len)

Coronovirus is a strong desease of CCC in CCC where it can be removed through the addition of antigens The nucleic acids found in this organism are derived from the cytosol in the cytoplasm which acts as a barrier to the cells it binds to the cytoplasm and eventually the enzyme phospholipids These phosphate groups act as markers for covalent bonds The structure of CCC can be distinguished by the fact that covalent bonds


In [26]:
sequence = "Coctails in space"
max_len = 50
generate_text(sequence, max_len)

Coctails in space are not used in the same way as a rocket engine In fact one can also see how this results in afreedom from gravity by placing the vehicle in space and then releasing a payload The final advantage to nventional propulsion is that
